# AI Agent 與工具使用 (AI Agents and Tool Use)

本 notebook 對應李宏毅老師 2025 Spring ML HW2，深入探討 AI Agent 的設計與工具使用能力。

## 學習目標

1. 理解 AI Agent 的核心概念
2. 掌握 ReAct 框架的原理與實作
3. 學習 Tool Use（Function Calling）的設計
4. 實作多步驟推理與工具調用
5. 了解多 Agent 系統的設計

## 參考資源

- [ReAct Paper](https://arxiv.org/abs/2210.03629) - Reasoning and Acting in Language Models
- [Toolformer](https://arxiv.org/abs/2302.04761) - LLM Learning to Use Tools
- [LangChain Agents](https://python.langchain.com/docs/modules/agents/)
- [2025 Spring HW2](https://speech.ee.ntu.edu.tw/~hylee/ml/2025-spring.php)

## 1. AI Agent 概念介紹

### 1.1 什麼是 AI Agent？

```
┌─────────────────────────────────────────────────────────────────────────┐
│                         AI Agent 定義                                    │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  傳統 LLM：                                                              │
│  ┌─────────┐                    ┌─────────┐                             │
│  │  Input  │───────→  LLM  ───→ │ Output  │                             │
│  │ (Query) │                    │ (Text)  │                             │
│  └─────────┘                    └─────────┘                             │
│  一次輸入 → 一次輸出，無法與外界互動                                       │
│                                                                         │
│  AI Agent：                                                             │
│  ┌─────────┐      ┌─────────────────────────────────┐                   │
│  │  Goal   │──→   │           Agent Core            │                   │
│  │  Task   │      │  ┌───────────────────────────┐  │                   │
│  └─────────┘      │  │    Planning & Reasoning   │  │                   │
│                   │  │         (LLM Brain)        │  │                   │
│                   │  └─────────────┬─────────────┘  │                   │
│                   │                │                 │                   │
│                   │    ┌───────────┼───────────┐    │                   │
│                   │    ▼           ▼           ▼    │                   │
│                   │ ┌─────┐   ┌─────────┐  ┌─────┐  │                   │
│                   │ │Tool │   │ Memory  │  │Tool │  │                   │
│                   │ │  A  │   │         │  │  B  │  │                   │
│                   │ └──┬──┘   └────┬────┘  └──┬──┘  │                   │
│                   └────┼──────────┼──────────┼─────┘                    │
│                        ▼          ▼          ▼                          │
│                   ┌─────────────────────────────────┐                   │
│                   │        External World           │                   │
│                   │  (APIs, Databases, Web, etc.)  │                   │
│                   └─────────────────────────────────┘                   │
│                                                                         │
│  關鍵特性：                                                              │
│  1. 自主決策（Autonomy）                                                 │
│  2. 與環境互動（Interaction）                                            │
│  3. 多步驟規劃（Planning）                                               │
│  4. 使用工具（Tool Use）                                                 │
│  5. 記憶與學習（Memory）                                                 │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
import json
import re
from typing import List, Dict, Callable, Any, Optional, Tuple
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
import random

# 設定隨機種子
random.seed(42)

print("AI Agent 基礎模組已載入")

## 2. Tool（工具）的定義與實作

### 2.1 Tool 的基本結構

每個工具需要包含：
1. **名稱**：唯一識別符
2. **描述**：讓 LLM 理解何時使用
3. **參數定義**：JSON Schema 格式
4. **執行函數**：實際執行邏輯

In [ ]:
@dataclass
class ToolParameter:
    """工具參數定義"""
    name: str
    type: str  # string, number, boolean, array, object
    description: str
    required: bool = True
    enum: Optional[List[str]] = None  # 可選值列表


@dataclass
class Tool:
    """工具定義"""
    name: str
    description: str
    parameters: List[ToolParameter]
    function: Callable[..., str]  # 執行函數
    
    def to_schema(self) -> Dict:
        """轉換為 OpenAI Function Calling 格式"""
        properties = {}
        required = []
        
        for param in self.parameters:
            prop = {"type": param.type, "description": param.description}
            if param.enum:
                prop["enum"] = param.enum
            properties[param.name] = prop
            if param.required:
                required.append(param.name)
        
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parameters": {
                    "type": "object",
                    "properties": properties,
                    "required": required
                }
            }
        }
    
    def execute(self, **kwargs) -> str:
        """執行工具"""
        return self.function(**kwargs)


# 範例工具 1: 計算器
def calculator(expression: str) -> str:
    """簡單計算器"""
    try:
        # 安全的數學運算（僅允許基本運算）
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

calculator_tool = Tool(
    name="calculator",
    description="Perform mathematical calculations. Use this when you need to compute arithmetic expressions.",
    parameters=[
        ToolParameter(
            name="expression",
            type="string",
            description="The mathematical expression to evaluate, e.g., '2 + 3 * 4'"
        )
    ],
    function=calculator
)

# 範例工具 2: 天氣查詢（模擬）
def get_weather(city: str, unit: str = "celsius") -> str:
    """模擬天氣 API"""
    # 模擬天氣資料
    weather_data = {
        "taipei": {"temp": 28, "condition": "Partly Cloudy", "humidity": 75},
        "tokyo": {"temp": 22, "condition": "Sunny", "humidity": 60},
        "new york": {"temp": 15, "condition": "Rainy", "humidity": 85},
        "london": {"temp": 12, "condition": "Cloudy", "humidity": 70},
    }
    
    city_lower = city.lower()
    if city_lower in weather_data:
        data = weather_data[city_lower]
        temp = data["temp"]
        if unit == "fahrenheit":
            temp = temp * 9/5 + 32
        return f"Weather in {city}: {data['condition']}, Temperature: {temp}°{'F' if unit == 'fahrenheit' else 'C'}, Humidity: {data['humidity']}%"
    else:
        return f"Weather data not available for {city}"

weather_tool = Tool(
    name="get_weather",
    description="Get current weather information for a city.",
    parameters=[
        ToolParameter(
            name="city",
            type="string",
            description="The city name to get weather for"
        ),
        ToolParameter(
            name="unit",
            type="string",
            description="Temperature unit",
            required=False,
            enum=["celsius", "fahrenheit"]
        )
    ],
    function=get_weather
)

# 範例工具 3: 搜尋（模擬）
def web_search(query: str, num_results: int = 3) -> str:
    """模擬網路搜尋"""
    # 模擬搜尋結果
    mock_results = {
        "python": [
            {"title": "Python Official Website", "url": "python.org", "snippet": "Python is a programming language..."},
            {"title": "Learn Python - Tutorial", "url": "learnpython.org", "snippet": "Free interactive Python tutorial..."},
            {"title": "Python Documentation", "url": "docs.python.org", "snippet": "Official Python documentation..."}
        ],
        "machine learning": [
            {"title": "ML Course by Andrew Ng", "url": "coursera.org/ml", "snippet": "Stanford's machine learning course..."},
            {"title": "Scikit-learn", "url": "scikit-learn.org", "snippet": "Simple and efficient ML tools..."},
            {"title": "Deep Learning Book", "url": "deeplearningbook.org", "snippet": "Comprehensive deep learning resource..."}
        ]
    }
    
    # 簡單的關鍵字匹配
    query_lower = query.lower()
    for key in mock_results:
        if key in query_lower:
            results = mock_results[key][:num_results]
            output = f"Search results for '{query}':\n"
            for i, r in enumerate(results, 1):
                output += f"{i}. {r['title']} ({r['url']})\n   {r['snippet']}\n"
            return output
    
    return f"No results found for '{query}'"

search_tool = Tool(
    name="web_search",
    description="Search the web for information. Use this when you need to find up-to-date information or facts.",
    parameters=[
        ToolParameter(
            name="query",
            type="string",
            description="The search query"
        ),
        ToolParameter(
            name="num_results",
            type="number",
            description="Number of results to return",
            required=False
        )
    ],
    function=web_search
)

# 展示工具定義
print("Calculator Tool Schema:")
print(json.dumps(calculator_tool.to_schema(), indent=2))
print("\nTest calculator:", calculator_tool.execute(expression="2 + 3 * 4"))
print("\nTest weather:", weather_tool.execute(city="Taipei"))
print("\nTest search:", search_tool.execute(query="machine learning"))

## 3. ReAct 框架

### 3.1 ReAct 原理

ReAct = **Rea**soning + **Act**ing

```
┌─────────────────────────────────────────────────────────────────────────┐
│                          ReAct 迴圈                                      │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  ┌─────────────────────────────────────────────────────────────────┐   │
│  │                        Agent Loop                               │   │
│  │                                                                 │   │
│  │    Question ─────→ Thought ─────→ Action ─────→ Observation    │   │
│  │        │              │              │              │          │   │
│  │        │              │              │              │          │   │
│  │        │              ▼              ▼              │          │   │
│  │        │         「我需要...」   Tool Call        │          │   │
│  │        │         「讓我想想」   or "Finish"       │          │   │
│  │        │                                           │          │   │
│  │        │                                           │          │   │
│  │        │         ◄──────── Repeat ────────────────┘          │   │
│  │        │                                                       │   │
│  │        └───────────────────────────────────────────────────────│   │
│  │                                                                 │   │
│  └─────────────────────────────────────────────────────────────────┘   │
│                                                                         │
│  範例：                                                                  │
│  ──────                                                                 │
│  Question: What is the weather in Tokyo and what's 25*4?               │
│                                                                         │
│  Thought 1: I need to find the weather in Tokyo first.                 │
│  Action 1: get_weather(city="Tokyo")                                   │
│  Observation 1: Weather in Tokyo: Sunny, 22°C                          │
│                                                                         │
│  Thought 2: Now I need to calculate 25*4.                              │
│  Action 2: calculator(expression="25*4")                               │
│  Observation 2: Result: 100                                            │
│                                                                         │
│  Thought 3: I have all the information needed.                         │
│  Action 3: Finish("Tokyo weather is Sunny at 22°C, and 25*4=100")     │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
@dataclass
class AgentAction:
    """Agent 的動作"""
    tool: str
    tool_input: Dict[str, Any]
    thought: str


@dataclass
class AgentFinish:
    """Agent 完成的回應"""
    output: str
    thought: str


class ReActAgent:
    """
    ReAct Agent 實作
    
    這是一個簡化版本，使用規則來模擬 LLM 的決策
    實際應用中會使用真正的 LLM 來產生 Thought 和 Action
    """
    def __init__(self, tools: List[Tool], max_iterations: int = 5):
        self.tools = {tool.name: tool for tool in tools}
        self.max_iterations = max_iterations
        self.scratchpad: List[str] = []  # 記錄思考過程
    
    def _get_tool_descriptions(self) -> str:
        """取得所有工具的描述"""
        descriptions = []
        for name, tool in self.tools.items():
            params = ", ".join([f"{p.name}: {p.type}" for p in tool.parameters])
            descriptions.append(f"- {name}({params}): {tool.description}")
        return "\n".join(descriptions)
    
    def _create_prompt(self, question: str) -> str:
        """建立 ReAct prompt"""
        prompt = f"""Answer the following question by reasoning step by step and using tools when needed.

Available tools:
{self._get_tool_descriptions()}

Use the following format:
Question: the input question
Thought: think about what to do
Action: the action to take (tool name)
Action Input: the input to the action (JSON format)
Observation: the result of the action
... (repeat Thought/Action/Observation as needed)
Thought: I now have enough information
Final Answer: the final answer to the question

Question: {question}
"""
        # 加入之前的思考過程
        if self.scratchpad:
            prompt += "\n".join(self.scratchpad) + "\n"
        
        return prompt
    
    def _parse_action(self, llm_output: str) -> Tuple[Optional[str], Optional[Dict], Optional[str]]:
        """
        解析 LLM 輸出，提取 action 和 action input
        
        Returns:
            (tool_name, tool_input, thought) 或 (None, None, final_answer)
        """
        # 檢查是否為最終答案
        if "Final Answer:" in llm_output:
            final_answer = llm_output.split("Final Answer:")[-1].strip()
            thought = llm_output.split("Final Answer:")[0].strip()
            return None, None, final_answer
        
        # 解析 Thought
        thought_match = re.search(r"Thought:\s*(.+?)(?=Action:|$)", llm_output, re.DOTALL)
        thought = thought_match.group(1).strip() if thought_match else ""
        
        # 解析 Action
        action_match = re.search(r"Action:\s*(\w+)", llm_output)
        if not action_match:
            return None, None, None
        tool_name = action_match.group(1)
        
        # 解析 Action Input
        input_match = re.search(r"Action Input:\s*({.+})", llm_output, re.DOTALL)
        if input_match:
            try:
                tool_input = json.loads(input_match.group(1))
            except json.JSONDecodeError:
                tool_input = {}
        else:
            tool_input = {}
        
        return tool_name, tool_input, thought
    
    def _simulate_llm_response(self, question: str, iteration: int) -> str:
        """
        模擬 LLM 回應（實際使用時替換為真正的 LLM 調用）
        
        這裡使用簡單的規則來模擬 LLM 的決策
        """
        question_lower = question.lower()
        
        # 第一次迭代：根據問題選擇工具
        if iteration == 0:
            if "weather" in question_lower:
                # 從問題中提取城市名
                cities = ["taipei", "tokyo", "new york", "london"]
                for city in cities:
                    if city in question_lower:
                        return f"""Thought: I need to find the weather information for {city.title()}.
Action: get_weather
Action Input: {{"city": "{city.title()}"}}"""
            
            if any(op in question_lower for op in ["+", "-", "*", "/", "calculate", "what is"]):
                # 嘗試提取數學表達式
                expr_match = re.search(r"what is ([\d\s\+\-\*\/\.\(\)]+)", question_lower)
                if expr_match:
                    expr = expr_match.group(1).strip()
                    return f"""Thought: I need to calculate the mathematical expression.
Action: calculator
Action Input: {{"expression": "{expr}"}}"""
            
            if "search" in question_lower or "find" in question_lower:
                return f"""Thought: I need to search for information.
Action: web_search
Action Input: {{"query": "{question}"}}"""
        
        # 後續迭代：檢查是否需要更多操作
        # 簡化處理：假設一次操作後就完成
        observations = "\n".join(self.scratchpad)
        return f"""Thought: I have gathered the necessary information from the tools.
Final Answer: Based on the observations, here is what I found: {observations}"""
    
    def run(self, question: str) -> str:
        """執行 Agent"""
        self.scratchpad = []  # 清空思考記錄
        
        print(f"Question: {question}")
        print("=" * 60)
        
        for iteration in range(self.max_iterations):
            print(f"\n--- Iteration {iteration + 1} ---")
            
            # 模擬 LLM 回應
            llm_response = self._simulate_llm_response(question, iteration)
            print(f"LLM Response:\n{llm_response}")
            
            # 解析回應
            tool_name, tool_input, thought_or_answer = self._parse_action(llm_response)
            
            # 如果是最終答案
            if tool_name is None and thought_or_answer is not None:
                print(f"\nFinal Answer: {thought_or_answer}")
                return thought_or_answer
            
            # 執行工具
            if tool_name and tool_name in self.tools:
                tool = self.tools[tool_name]
                observation = tool.execute(**tool_input)
                print(f"\nObservation: {observation}")
                
                # 記錄到 scratchpad
                self.scratchpad.append(f"Thought: {thought_or_answer if thought_or_answer else ''}")
                self.scratchpad.append(f"Action: {tool_name}")
                self.scratchpad.append(f"Action Input: {json.dumps(tool_input)}")
                self.scratchpad.append(f"Observation: {observation}")
            else:
                print(f"\nError: Unknown tool '{tool_name}'")
                break
        
        return "Max iterations reached without finding an answer."


# 建立 Agent
react_agent = ReActAgent(
    tools=[calculator_tool, weather_tool, search_tool],
    max_iterations=5
)

print("ReAct Agent 已建立！")
print(f"\n可用工具: {list(react_agent.tools.keys())}")

In [ ]:
# 測試 ReAct Agent
print("\n" + "="*60)
print("測試 1: 天氣查詢")
print("="*60)
result1 = react_agent.run("What is the weather in Tokyo?")

print("\n" + "="*60)
print("測試 2: 數學計算")
print("="*60)
result2 = react_agent.run("What is 25 * 4 + 10?")

## 4. Function Calling（工具調用）

### 4.1 OpenAI Function Calling 格式

```python
# OpenAI API Function Calling 範例
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather for a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "City name"
                    }
                },
                "required": ["city"]
            }
        }
    }
]

# LLM 回應會包含 tool_calls
response = {
    "tool_calls": [
        {
            "id": "call_123",
            "type": "function",
            "function": {
                "name": "get_weather",
                "arguments": '{"city": "Tokyo"}'
            }
        }
    ]
}
```

In [ ]:
class FunctionCallingAgent:
    """
    模擬 OpenAI Function Calling 風格的 Agent
    """
    def __init__(self, tools: List[Tool]):
        self.tools = {tool.name: tool for tool in tools}
        self.message_history: List[Dict] = []
    
    def get_tools_schema(self) -> List[Dict]:
        """取得所有工具的 schema"""
        return [tool.to_schema() for tool in self.tools.values()]
    
    def _simulate_llm_function_call(self, user_message: str) -> Dict:
        """
        模擬 LLM 決定是否調用函數
        實際使用時會調用 OpenAI API
        """
        message_lower = user_message.lower()
        
        # 檢測是否需要調用工具
        if "weather" in message_lower:
            cities = ["taipei", "tokyo", "new york", "london"]
            for city in cities:
                if city in message_lower:
                    return {
                        "role": "assistant",
                        "content": None,
                        "tool_calls": [{
                            "id": f"call_{random.randint(1000, 9999)}",
                            "type": "function",
                            "function": {
                                "name": "get_weather",
                                "arguments": json.dumps({"city": city.title()})
                            }
                        }]
                    }
        
        if any(op in message_lower for op in ["calculate", "what is", "+", "-", "*", "/"]):
            expr_match = re.search(r"what is ([\d\s\+\-\*\/\.\(\)]+)", message_lower)
            if expr_match:
                expr = expr_match.group(1).strip()
                return {
                    "role": "assistant",
                    "content": None,
                    "tool_calls": [{
                        "id": f"call_{random.randint(1000, 9999)}",
                        "type": "function",
                        "function": {
                            "name": "calculator",
                            "arguments": json.dumps({"expression": expr})
                        }
                    }]
                }
        
        # 沒有需要的工具，直接回應
        return {
            "role": "assistant",
            "content": f"I don't need any tools to answer this. Here's my response: I understand your question about '{user_message}'."
        }
    
    def _execute_tool_calls(self, tool_calls: List[Dict]) -> List[Dict]:
        """執行工具調用"""
        results = []
        for call in tool_calls:
            tool_name = call["function"]["name"]
            arguments = json.loads(call["function"]["arguments"])
            
            if tool_name in self.tools:
                result = self.tools[tool_name].execute(**arguments)
            else:
                result = f"Error: Unknown tool '{tool_name}'"
            
            results.append({
                "role": "tool",
                "tool_call_id": call["id"],
                "name": tool_name,
                "content": result
            })
        
        return results
    
    def chat(self, user_message: str) -> str:
        """處理使用者訊息"""
        # 加入使用者訊息
        self.message_history.append({
            "role": "user",
            "content": user_message
        })
        
        print(f"User: {user_message}")
        
        # 模擬 LLM 回應
        assistant_message = self._simulate_llm_function_call(user_message)
        self.message_history.append(assistant_message)
        
        # 如果有工具調用
        if assistant_message.get("tool_calls"):
            print(f"\nAssistant decides to call tools:")
            for call in assistant_message["tool_calls"]:
                print(f"  - {call['function']['name']}({call['function']['arguments']})")
            
            # 執行工具
            tool_results = self._execute_tool_calls(assistant_message["tool_calls"])
            self.message_history.extend(tool_results)
            
            print(f"\nTool results:")
            for result in tool_results:
                print(f"  - {result['name']}: {result['content']}")
            
            # 模擬最終回應（實際會再次調用 LLM）
            final_response = f"Based on the tool results: {tool_results[0]['content']}"
            print(f"\nAssistant: {final_response}")
            return final_response
        else:
            print(f"\nAssistant: {assistant_message['content']}")
            return assistant_message["content"]


# 建立 Function Calling Agent
fc_agent = FunctionCallingAgent(tools=[calculator_tool, weather_tool, search_tool])

print("Function Calling Agent Tools Schema:")
print(json.dumps(fc_agent.get_tools_schema()[:1], indent=2))  # 只顯示第一個

In [ ]:
# 測試 Function Calling Agent
print("\n" + "="*60)
print("測試 Function Calling Agent")
print("="*60 + "\n")

fc_agent.chat("What is the weather in London?")
print("\n" + "-"*40 + "\n")
fc_agent.chat("What is 123 * 456?")

## 5. Agent 記憶系統

### 5.1 記憶類型

```
┌─────────────────────────────────────────────────────────────────────────┐
│                       Agent 記憶系統                                     │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  1. 短期記憶 (Short-term Memory)                                        │
│     ┌─────────────────────────────────────────────────────────────┐    │
│     │  • 當前對話的上下文                                           │    │
│     │  • 最近的 Thought/Action/Observation                         │    │
│     │  • 通常存在 prompt 中                                         │    │
│     └─────────────────────────────────────────────────────────────┘    │
│                                                                         │
│  2. 長期記憶 (Long-term Memory)                                         │
│     ┌─────────────────────────────────────────────────────────────┐    │
│     │  • 向量資料庫儲存                                             │    │
│     │  • 過往對話的重要資訊                                         │    │
│     │  • 使用者偏好和知識                                           │    │
│     └─────────────────────────────────────────────────────────────┘    │
│                                                                         │
│  3. 工作記憶 (Working Memory)                                           │
│     ┌─────────────────────────────────────────────────────────────┐    │
│     │  • 當前任務的狀態                                             │    │
│     │  • 中間計算結果                                               │    │
│     │  • Scratchpad                                                │    │
│     └─────────────────────────────────────────────────────────────┘    │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from collections import deque
from datetime import datetime


@dataclass
class MemoryItem:
    """記憶項目"""
    content: str
    timestamp: datetime = field(default_factory=datetime.now)
    importance: float = 0.5  # 重要性分數
    type: str = "observation"  # thought, action, observation, user_input


class AgentMemory:
    """
    Agent 記憶系統
    """
    def __init__(self, 
                 short_term_capacity: int = 10,
                 working_memory_capacity: int = 5):
        # 短期記憶：最近的對話
        self.short_term = deque(maxlen=short_term_capacity)
        
        # 工作記憶：當前任務相關
        self.working_memory = deque(maxlen=working_memory_capacity)
        
        # 長期記憶：重要資訊（簡化版，實際會用向量資料庫）
        self.long_term: List[MemoryItem] = []
        
        # 重要性閾值
        self.importance_threshold = 0.7
    
    def add(self, content: str, type: str = "observation", importance: float = 0.5):
        """新增記憶"""
        item = MemoryItem(content=content, type=type, importance=importance)
        
        # 加入短期記憶
        self.short_term.append(item)
        
        # 如果重要性高，也加入長期記憶
        if importance >= self.importance_threshold:
            self.long_term.append(item)
    
    def add_to_working(self, content: str, type: str = "observation"):
        """新增到工作記憶"""
        item = MemoryItem(content=content, type=type)
        self.working_memory.append(item)
    
    def clear_working(self):
        """清空工作記憶"""
        self.working_memory.clear()
    
    def get_context(self, max_items: int = 5) -> str:
        """取得上下文供 prompt 使用"""
        context_parts = []
        
        # 工作記憶
        if self.working_memory:
            context_parts.append("Current task context:")
            for item in self.working_memory:
                context_parts.append(f"  [{item.type}] {item.content}")
        
        # 最近的短期記憶
        recent = list(self.short_term)[-max_items:]
        if recent:
            context_parts.append("\nRecent conversation:")
            for item in recent:
                context_parts.append(f"  [{item.type}] {item.content}")
        
        return "\n".join(context_parts)
    
    def search_long_term(self, query: str, top_k: int = 3) -> List[MemoryItem]:
        """
        搜尋長期記憶（簡化版，使用關鍵字匹配）
        實際應用會使用向量相似度搜尋
        """
        query_words = set(query.lower().split())
        scores = []
        
        for item in self.long_term:
            content_words = set(item.content.lower().split())
            overlap = len(query_words & content_words)
            scores.append((item, overlap))
        
        # 排序並返回前 k 個
        scores.sort(key=lambda x: x[1], reverse=True)
        return [item for item, score in scores[:top_k] if score > 0]


# 測試記憶系統
memory = AgentMemory()

# 模擬對話
memory.add("User asked about Tokyo weather", type="user_input")
memory.add_to_working("Need to call weather API", type="thought")
memory.add("Weather in Tokyo: Sunny, 22°C", type="observation")
memory.add("User seems interested in Japan travel", type="thought", importance=0.8)

print("Agent Memory Context:")
print(memory.get_context())

print("\n\nLong-term memories:")
for item in memory.long_term:
    print(f"  - {item.content} (importance: {item.importance})")

## 6. 多 Agent 系統

### 6.1 Multi-Agent 架構

```
┌─────────────────────────────────────────────────────────────────────────┐
│                       Multi-Agent 系統                                   │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│                         ┌─────────────┐                                 │
│                         │ Orchestrator│                                 │
│                         │   Agent     │                                 │
│                         └──────┬──────┘                                 │
│                                │                                        │
│              ┌─────────────────┼─────────────────┐                      │
│              │                 │                 │                      │
│              ▼                 ▼                 ▼                      │
│       ┌──────────┐      ┌──────────┐      ┌──────────┐                 │
│       │ Research │      │  Writer  │      │ Reviewer │                 │
│       │  Agent   │      │  Agent   │      │  Agent   │                 │
│       └────┬─────┘      └────┬─────┘      └────┬─────┘                 │
│            │                 │                 │                        │
│            ▼                 ▼                 ▼                        │
│       ┌─────────┐       ┌─────────┐       ┌─────────┐                  │
│       │ Search  │       │ Draft   │       │ Feedback│                  │
│       │ Tools   │       │ Tools   │       │ Tools   │                  │
│       └─────────┘       └─────────┘       └─────────┘                  │
│                                                                         │
│  優點：                                                                  │
│  1. 專業化分工                                                           │
│  2. 可以平行執行                                                         │
│  3. 更容易除錯和改進                                                     │
│  4. 可以混合不同模型                                                     │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
class BaseAgent(ABC):
    """Agent 基礎類別"""
    def __init__(self, name: str, role: str):
        self.name = name
        self.role = role
    
    @abstractmethod
    def process(self, input_data: Dict) -> Dict:
        """處理輸入並返回輸出"""
        pass


class ResearchAgent(BaseAgent):
    """研究 Agent - 負責收集資訊"""
    def __init__(self):
        super().__init__("Researcher", "Gather information and facts")
    
    def process(self, input_data: Dict) -> Dict:
        topic = input_data.get("topic", "")
        print(f"[{self.name}] Researching: {topic}")
        
        # 模擬研究結果
        research_results = {
            "topic": topic,
            "facts": [
                f"Fact 1 about {topic}",
                f"Fact 2 about {topic}",
                f"Fact 3 about {topic}"
            ],
            "sources": ["source1.com", "source2.org"]
        }
        
        print(f"[{self.name}] Found {len(research_results['facts'])} facts")
        return research_results


class WriterAgent(BaseAgent):
    """寫作 Agent - 負責撰寫內容"""
    def __init__(self):
        super().__init__("Writer", "Write content based on research")
    
    def process(self, input_data: Dict) -> Dict:
        research = input_data.get("research", {})
        style = input_data.get("style", "informative")
        
        print(f"[{self.name}] Writing {style} content...")
        
        # 模擬寫作結果
        facts = research.get("facts", [])
        content = f"""# {research.get('topic', 'Topic')}

Based on our research, here are the key findings:

""" + "\n".join([f"- {fact}" for fact in facts])
        
        print(f"[{self.name}] Draft completed ({len(content)} chars)")
        return {"draft": content, "word_count": len(content.split())}


class ReviewerAgent(BaseAgent):
    """審核 Agent - 負責審核和改進"""
    def __init__(self):
        super().__init__("Reviewer", "Review and provide feedback")
    
    def process(self, input_data: Dict) -> Dict:
        draft = input_data.get("draft", "")
        
        print(f"[{self.name}] Reviewing draft...")
        
        # 模擬審核結果
        feedback = {
            "approved": True,
            "score": 8.5,
            "suggestions": [
                "Consider adding more examples",
                "The conclusion could be stronger"
            ]
        }
        
        print(f"[{self.name}] Review complete. Score: {feedback['score']}/10")
        return feedback


class OrchestratorAgent:
    """
    協調者 Agent - 管理多個 Agent 的協作
    """
    def __init__(self):
        self.agents = {
            "researcher": ResearchAgent(),
            "writer": WriterAgent(),
            "reviewer": ReviewerAgent()
        }
    
    def execute_workflow(self, task: str) -> Dict:
        """
        執行完整的工作流程
        Research → Write → Review
        """
        print(f"\n{'='*60}")
        print(f"Starting workflow for: {task}")
        print(f"{'='*60}\n")
        
        # Step 1: Research
        print("Step 1: Research Phase")
        print("-" * 40)
        research_results = self.agents["researcher"].process({"topic": task})
        
        # Step 2: Write
        print("\nStep 2: Writing Phase")
        print("-" * 40)
        writing_results = self.agents["writer"].process({
            "research": research_results,
            "style": "informative"
        })
        
        # Step 3: Review
        print("\nStep 3: Review Phase")
        print("-" * 40)
        review_results = self.agents["reviewer"].process({
            "draft": writing_results["draft"]
        })
        
        # 彙整結果
        final_output = {
            "task": task,
            "research": research_results,
            "content": writing_results["draft"],
            "review": review_results
        }
        
        print(f"\n{'='*60}")
        print("Workflow completed!")
        print(f"{'='*60}")
        
        return final_output


# 測試多 Agent 系統
orchestrator = OrchestratorAgent()
result = orchestrator.execute_workflow("Artificial Intelligence in Healthcare")

print("\n\nFinal Content:")
print(result["content"])

## 7. 進階主題：Planning 策略

### 7.1 不同的 Planning 方法

```
┌─────────────────────────────────────────────────────────────────────────┐
│                       Planning 策略比較                                  │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  1. ReAct (Reasoning + Acting)                                         │
│     ─────────────────────────                                          │
│     • 交錯進行 Thought 和 Action                                        │
│     • 每步都基於觀察調整                                                 │
│     • 靈活但可能不穩定                                                   │
│                                                                         │
│  2. Plan-and-Execute                                                   │
│     ───────────────────                                                │
│     • 先產生完整計畫                                                    │
│     • 再依序執行                                                        │
│     • 穩定但不夠靈活                                                    │
│                                                                         │
│  3. Tree of Thoughts (ToT)                                             │
│     ─────────────────────                                              │
│     • 探索多個思考路徑                                                  │
│     • 評估每個分支                                                      │
│     • 選擇最佳路徑                                                      │
│                                                                         │
│  4. Reflexion                                                          │
│     ──────────                                                         │
│     • 執行後反思                                                        │
│     • 從錯誤學習                                                        │
│     • 持續改進                                                          │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
class PlanAndExecuteAgent:
    """
    Plan-and-Execute Agent
    先產生計畫，再依序執行
    """
    def __init__(self, tools: List[Tool]):
        self.tools = {tool.name: tool for tool in tools}
    
    def create_plan(self, goal: str) -> List[Dict]:
        """
        根據目標建立執行計畫
        實際使用時會用 LLM 產生
        """
        print(f"Creating plan for goal: {goal}")
        
        # 模擬 LLM 產生計畫
        goal_lower = goal.lower()
        plan = []
        
        # 根據關鍵字生成計畫步驟
        if "weather" in goal_lower:
            cities = ["taipei", "tokyo", "london"]
            for city in cities:
                if city in goal_lower:
                    plan.append({
                        "step": len(plan) + 1,
                        "description": f"Get weather for {city.title()}",
                        "tool": "get_weather",
                        "args": {"city": city.title()}
                    })
        
        if "calculate" in goal_lower or any(op in goal_lower for op in ["+", "-", "*", "/"]):
            expr_match = re.search(r"([\d\s\+\-\*\/\.\(\)]+)", goal)
            if expr_match:
                plan.append({
                    "step": len(plan) + 1,
                    "description": f"Calculate expression",
                    "tool": "calculator",
                    "args": {"expression": expr_match.group(1).strip()}
                })
        
        # 加入彙總步驟
        plan.append({
            "step": len(plan) + 1,
            "description": "Summarize findings",
            "tool": None,
            "args": {}
        })
        
        return plan
    
    def execute_plan(self, plan: List[Dict]) -> Dict:
        """
        執行計畫中的每個步驟
        """
        results = []
        
        print("\n" + "="*50)
        print("Executing Plan")
        print("="*50)
        
        for step in plan:
            print(f"\nStep {step['step']}: {step['description']}")
            
            if step["tool"] and step["tool"] in self.tools:
                tool = self.tools[step["tool"]]
                result = tool.execute(**step["args"])
                print(f"  → Result: {result}")
                results.append({
                    "step": step["step"],
                    "description": step["description"],
                    "result": result
                })
            else:
                # 彙總步驟
                summary = "Summary: " + "; ".join([r["result"] for r in results])
                print(f"  → {summary}")
                results.append({
                    "step": step["step"],
                    "description": step["description"],
                    "result": summary
                })
        
        return {
            "plan": plan,
            "results": results,
            "final_answer": results[-1]["result"] if results else "No results"
        }
    
    def run(self, goal: str) -> str:
        """執行完整的 Plan-and-Execute 流程"""
        # Phase 1: Planning
        plan = self.create_plan(goal)
        
        print("\nGenerated Plan:")
        for step in plan:
            print(f"  {step['step']}. {step['description']}")
        
        # Phase 2: Execution
        results = self.execute_plan(plan)
        
        return results["final_answer"]


# 測試 Plan-and-Execute Agent
pe_agent = PlanAndExecuteAgent(tools=[calculator_tool, weather_tool, search_tool])
result = pe_agent.run("Get the weather in Tokyo and calculate 15 * 8")

## 8. 練習題

### 練習 1：建立自定義工具

In [ ]:
# 練習 1：建立一個新的工具 - 日期時間工具

def get_current_datetime(timezone: str = "UTC", format: str = "full") -> str:
    """
    TODO: 實作取得當前日期時間的函數
    
    Args:
        timezone: 時區 (UTC, Asia/Taipei, America/New_York, etc.)
        format: 輸出格式 (full, date, time)
    
    Returns:
        格式化的日期時間字串
    """
    # 提示：可以使用 datetime 模組
    pass


# TODO: 建立 datetime_tool
# datetime_tool = Tool(
#     name="get_datetime",
#     description="...",
#     parameters=[...],
#     function=get_current_datetime
# )

print("練習 1：實作 get_current_datetime 函數並建立 Tool 物件")

### 練習 2：實作帶有記憶的 Agent

In [ ]:
# 練習 2：實作帶有記憶的對話 Agent

class ConversationalAgent:
    """
    TODO: 實作一個可以記住對話歷史的 Agent
    
    功能：
    1. 記住使用者的名字和偏好
    2. 可以引用之前的對話內容
    3. 結合工具使用
    """
    def __init__(self, tools: List[Tool]):
        self.tools = {tool.name: tool for tool in tools}
        self.memory = AgentMemory()
        self.user_profile = {}  # 儲存使用者資訊
    
    def extract_user_info(self, message: str):
        """
        TODO: 從訊息中提取使用者資訊
        例如："My name is John" → self.user_profile["name"] = "John"
        """
        pass
    
    def chat(self, message: str) -> str:
        """
        TODO: 處理使用者訊息
        1. 提取使用者資訊
        2. 儲存到記憶
        3. 決定是否使用工具
        4. 生成回應
        """
        pass

print("練習 2：完成 ConversationalAgent 的實作")

### 練習 3：建立簡單的 Code Agent

In [ ]:
# 練習 3：建立可以執行 Python 程式碼的 Agent

def execute_python(code: str) -> str:
    """
    安全地執行 Python 程式碼（受限環境）
    
    TODO: 實作此函數
    注意：需要處理安全性問題！
    """
    # 提示：
    # 1. 使用 exec() 或 eval()
    # 2. 限制可用的內建函數
    # 3. 設定執行時間限制
    # 4. 捕獲輸出
    pass


class CodeAgent:
    """
    TODO: 實作一個可以寫程式並執行的 Agent
    
    工作流程：
    1. 理解使用者的程式需求
    2. 生成 Python 程式碼
    3. 執行程式碼
    4. 返回結果或修正錯誤
    """
    def __init__(self):
        self.code_history = []
    
    def generate_code(self, task: str) -> str:
        """
        TODO: 根據任務生成程式碼
        實際使用時會用 LLM
        """
        pass
    
    def run(self, task: str) -> str:
        """
        TODO: 完整的執行流程
        """
        pass

print("練習 3：完成 Code Agent 的實作")

## 9. 總結

### 9.1 AI Agent 核心概念回顧

```
┌─────────────────────────────────────────────────────────────────────────┐
│                      AI Agent 關鍵要素                                   │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  1. LLM Core（大腦）                                                     │
│     • 理解自然語言指令                                                   │
│     • 推理和規劃能力                                                     │
│     • 決定何時使用哪個工具                                               │
│                                                                         │
│  2. Tools（工具）                                                        │
│     • 與外部系統互動                                                     │
│     • 擴展 LLM 能力（計算、搜尋、API）                                   │
│     • 需要清晰的描述和參數定義                                           │
│                                                                         │
│  3. Memory（記憶）                                                       │
│     • 短期：當前對話上下文                                               │
│     • 長期：向量資料庫儲存                                               │
│     • 工作記憶：任務狀態                                                 │
│                                                                         │
│  4. Planning（規劃）                                                     │
│     • ReAct：交錯思考和行動                                             │
│     • Plan-and-Execute：先規劃後執行                                    │
│     • 多 Agent 協作                                                     │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### 9.2 實際應用建議

| 情境 | 建議方法 |
|------|----------|
| 簡單工具調用 | Function Calling |
| 多步驟推理 | ReAct |
| 複雜工作流程 | Multi-Agent |
| 需要記憶 | 結合向量資料庫 |
| 需要反思改進 | Reflexion |

In [ ]:
print("="*60)
print("AI Agent 與工具使用 - 學習完成！")
print("="*60)
print("\n你已經學會：")
print("✓ AI Agent 的核心概念")
print("✓ Tool 的定義與實作")
print("✓ ReAct 框架的原理")
print("✓ Function Calling 的使用")
print("✓ Agent 記憶系統")
print("✓ 多 Agent 系統設計")
print("\n下一步學習建議：")
print("1. 使用 LangChain 或 AutoGen 建立真實 Agent")
print("2. 整合真正的 LLM（OpenAI, Claude, Llama）")
print("3. 學習更多 Planning 策略（ToT, Reflexion）")
print("4. 探索 Agent 安全性和可靠性")